# Chapter 15: Behavioral Patterns

This notebook covers behavioral design patterns that define how objects communicate and
distribute responsibilities. Python's first-class functions, iterator protocol, and context
managers make many of these patterns more elegant than their traditional OOP implementations.

## Key Concepts
- **Strategy pattern**: Swappable algorithms using first-class functions
- **Observer pattern**: Event-driven communication with EventBus
- **Iterator pattern**: Python's `__iter__` and `__next__` protocol
- **Command pattern**: Encapsulating actions as objects
- **Template Method pattern**: Defining algorithm skeleton in a base class
- **Protocol-based structural typing**: Duck typing with type safety
- **Context managers**: Resource management as a pattern
- When patterns help vs when they over-engineer

## Strategy Pattern Using First-Class Functions

The Strategy pattern lets you swap algorithms at runtime. In Java, this requires a Strategy
interface and concrete classes. In Python, **functions are objects** - just pass a different
function.

In [ ]:
from typing import Callable


# Define sorting strategies as plain functions
def sort_by_name(items: list[dict]) -> list[dict]:
    return sorted(items, key=lambda x: x["name"])


def sort_by_price(items: list[dict]) -> list[dict]:
    return sorted(items, key=lambda x: x["price"])


def sort_by_price_desc(items: list[dict]) -> list[dict]:
    return sorted(items, key=lambda x: x["price"], reverse=True)


# The "context" that uses a strategy
def display_products(
    items: list[dict],
    strategy: Callable[[list[dict]], list[dict]],
) -> None:
    sorted_items = strategy(items)
    for item in sorted_items:
        print(f"  {item['name']:10s} ${item['price']:.2f}")


products = [
    {"name": "Banana", "price": 1.50},
    {"name": "Apple", "price": 2.00},
    {"name": "Cherry", "price": 3.00},
    {"name": "Date", "price": 5.50},
]

print("Sorted by name:")
display_products(products, sort_by_name)

print("\nSorted by price (ascending):")
display_products(products, sort_by_price)

print("\nSorted by price (descending):")
display_products(products, sort_by_price_desc)

# You can even use a lambda as an inline strategy
print("\nSorted by name length (lambda):")
display_products(products, lambda items: sorted(items, key=lambda x: len(x["name"])))

## Observer Pattern with EventBus

The Observer pattern defines a one-to-many dependency: when one object changes state, all
its dependents are notified. An **EventBus** (or event emitter) is a clean way to decouple
publishers from subscribers.

In [ ]:
from typing import Callable, Any


class EventBus:
    """Simple publish-subscribe event system."""

    def __init__(self) -> None:
        self._handlers: dict[str, list[Callable]] = {}

    def subscribe(self, event: str, handler: Callable) -> None:
        """Register a handler for an event."""
        self._handlers.setdefault(event, []).append(handler)

    def unsubscribe(self, event: str, handler: Callable) -> None:
        """Remove a handler for an event."""
        if event in self._handlers:
            self._handlers[event] = [
                h for h in self._handlers[event] if h is not handler
            ]

    def emit(self, event: str, data: Any = None) -> None:
        """Notify all handlers registered for this event."""
        for handler in self._handlers.get(event, []):
            handler(data)


# Create an event bus
bus = EventBus()

# Track what happens
log: list[str] = []

# Subscribe handlers
bus.subscribe("user_created", lambda data: log.append(f"Send welcome email to {data}"))
bus.subscribe("user_created", lambda data: log.append(f"Log: new user {data}"))
bus.subscribe("user_created", lambda data: log.append(f"Analytics: track signup {data}"))
bus.subscribe("order_placed", lambda data: log.append(f"Process order for {data}"))

# Emit events
bus.emit("user_created", "Alice")
bus.emit("user_created", "Bob")
bus.emit("order_placed", "Alice")

print("Event log:")
for entry in log:
    print(f"  {entry}")

# Handlers are completely decoupled from the emitter
print(f"\nTotal events processed: {len(log)}")

## Iterator Pattern (Python's Iterator Protocol)

Python has built-in support for the Iterator pattern through `__iter__` and `__next__`.
Any object that implements these methods works with `for` loops, `list()`, unpacking, and
all other iteration contexts.

In [ ]:
class Countdown:
    """Custom iterator that counts down from a starting number."""

    def __init__(self, start: int) -> None:
        self.current = start

    def __iter__(self) -> "Countdown":
        return self

    def __next__(self) -> int:
        if self.current < 0:
            raise StopIteration
        value = self.current
        self.current -= 1
        return value


print("Countdown from 5:")
for n in Countdown(5):
    print(f"  {n}", end="")
print()


# Iterable (returns a new iterator each time) vs Iterator (single-use)
class FibonacciSequence:
    """Iterable that generates Fibonacci numbers up to a limit."""

    def __init__(self, max_value: int) -> None:
        self.max_value = max_value

    def __iter__(self):
        """Return a fresh iterator each time."""
        a, b = 0, 1
        while a <= self.max_value:
            yield a
            a, b = b, a + b


fib = FibonacciSequence(50)

# Can iterate multiple times because __iter__ returns a new generator each time
print(f"\nFibonacci up to 50: {list(fib)}")
print(f"Again:              {list(fib)}")

# Works with all iteration tools
print(f"Sum: {sum(fib)}")
print(f"Count: {len(list(fib))}")
print(f"Max: {max(fib)}")

## Command Pattern

The Command pattern encapsulates actions as objects, enabling undo/redo, queuing, and logging
of operations. Each command knows how to execute and optionally undo itself.

In [ ]:
from abc import ABC, abstractmethod


class Command(ABC):
    @abstractmethod
    def execute(self) -> str: ...

    @abstractmethod
    def undo(self) -> str: ...


class TextDocument:
    """Receiver - the object commands operate on."""

    def __init__(self) -> None:
        self.content: str = ""

    def __repr__(self) -> str:
        return f"Document({self.content!r})"


class InsertTextCommand(Command):
    def __init__(self, doc: TextDocument, text: str) -> None:
        self._doc = doc
        self._text = text

    def execute(self) -> str:
        self._doc.content += self._text
        return f"Inserted: {self._text!r}"

    def undo(self) -> str:
        self._doc.content = self._doc.content[: -len(self._text)]
        return f"Undid insert of: {self._text!r}"


class DeleteTextCommand(Command):
    def __init__(self, doc: TextDocument, count: int) -> None:
        self._doc = doc
        self._count = count
        self._deleted: str = ""

    def execute(self) -> str:
        self._deleted = self._doc.content[-self._count :]
        self._doc.content = self._doc.content[: -self._count]
        return f"Deleted: {self._deleted!r}"

    def undo(self) -> str:
        self._doc.content += self._deleted
        return f"Restored: {self._deleted!r}"


class CommandHistory:
    """Invoker - manages command execution and undo stack."""

    def __init__(self) -> None:
        self._history: list[Command] = []

    def execute(self, command: Command) -> str:
        result = command.execute()
        self._history.append(command)
        return result

    def undo(self) -> str:
        if not self._history:
            return "Nothing to undo"
        command = self._history.pop()
        return command.undo()


# Usage
doc = TextDocument()
history = CommandHistory()

print(history.execute(InsertTextCommand(doc, "Hello")))
print(f"  Doc: {doc}")

print(history.execute(InsertTextCommand(doc, " World")))
print(f"  Doc: {doc}")

print(history.execute(InsertTextCommand(doc, "!")))
print(f"  Doc: {doc}")

print(history.execute(DeleteTextCommand(doc, 6)))
print(f"  Doc: {doc}")

# Undo operations
print(f"\n{history.undo()}")
print(f"  Doc: {doc}")

print(history.undo())
print(f"  Doc: {doc}")

## Template Method Pattern Using ABC

The Template Method defines the **skeleton of an algorithm** in a base class, letting subclasses
override specific steps without changing the overall structure.

In [ ]:
from abc import ABC, abstractmethod


class DataPipeline(ABC):
    """Template: defines the algorithm skeleton."""

    def run(self, raw_data: list[str]) -> list[str]:
        """The template method - defines the steps, not the details."""
        data = self.extract(raw_data)
        data = self.transform(data)
        data = self.validate(data)
        return self.load(data)

    @abstractmethod
    def extract(self, raw: list[str]) -> list[str]: ...

    @abstractmethod
    def transform(self, data: list[str]) -> list[str]: ...

    def validate(self, data: list[str]) -> list[str]:
        """Default validation: remove empty strings. Subclasses can override."""
        return [d for d in data if d.strip()]

    @abstractmethod
    def load(self, data: list[str]) -> list[str]: ...


class UpperCasePipeline(DataPipeline):
    """Concrete pipeline: uppercase transformation."""

    def extract(self, raw: list[str]) -> list[str]:
        print("  [UpperCase] Extracting...")
        return [line.strip() for line in raw]

    def transform(self, data: list[str]) -> list[str]:
        print("  [UpperCase] Transforming to uppercase...")
        return [d.upper() for d in data]

    def load(self, data: list[str]) -> list[str]:
        print(f"  [UpperCase] Loaded {len(data)} records")
        return data


class CsvPipeline(DataPipeline):
    """Concrete pipeline: CSV-style formatting."""

    def extract(self, raw: list[str]) -> list[str]:
        print("  [CSV] Extracting and splitting...")
        return [line.strip() for line in raw]

    def transform(self, data: list[str]) -> list[str]:
        print("  [CSV] Wrapping in quotes...")
        return [f'"{d}"' for d in data]

    def validate(self, data: list[str]) -> list[str]:
        """Custom validation: also remove quoted empty strings."""
        print("  [CSV] Custom validation...")
        return [d for d in data if d != '""' and d.strip()]

    def load(self, data: list[str]) -> list[str]:
        result = ",".join(data)
        print(f"  [CSV] Loaded as: {result}")
        return data


raw_input = ["  alice  ", "bob", "", "charlie  ", "  "]

print("Running UpperCase pipeline:")
result1 = UpperCasePipeline().run(raw_input)
print(f"  Result: {result1}")

print("\nRunning CSV pipeline:")
result2 = CsvPipeline().run(raw_input)
print(f"  Result: {result2}")

## Protocol-Based Structural Typing

Python's `Protocol` formalizes duck typing: a class satisfies a Protocol if it has the right
methods, without needing to inherit from it. This is ideal for defining interfaces that
third-party code can satisfy without modification.

In [ ]:
from typing import Protocol, runtime_checkable


@runtime_checkable
class Renderable(Protocol):
    """Any object that can render itself to a string."""
    def render(self) -> str: ...


# These classes don't inherit from Renderable - they just match its shape
class HtmlWidget:
    def __init__(self, tag: str, content: str) -> None:
        self.tag = tag
        self.content = content

    def render(self) -> str:
        return f"<{self.tag}>{self.content}</{self.tag}>"


class JsonResponse:
    def __init__(self, data: dict) -> None:
        self.data = data

    def render(self) -> str:
        import json
        return json.dumps(self.data, indent=2)


class MarkdownHeading:
    def __init__(self, level: int, text: str) -> None:
        self.level = level
        self.text = text

    def render(self) -> str:
        return f"{'#' * self.level} {self.text}"


def render_all(items: list[Renderable]) -> str:
    """Works with ANY object that has a render() -> str method."""
    return "\n".join(item.render() for item in items)


components: list[Renderable] = [
    HtmlWidget("div", "Hello World"),
    JsonResponse({"status": "ok", "count": 42}),
    MarkdownHeading(2, "Section Title"),
]

print("Rendered output:")
print(render_all(components))

# runtime_checkable allows isinstance checks
print(f"\nHtmlWidget is Renderable: {isinstance(HtmlWidget('p', 'hi'), Renderable)}")
print(f"str is Renderable: {isinstance('hello', Renderable)}")

## Context Managers as Resource Management Pattern

Python's `with` statement and context managers implement the **Resource Acquisition Is
Initialization (RAII)** pattern. The `contextlib` module provides tools for creating
context managers without writing a full class.

In [ ]:
from contextlib import contextmanager
import time


# Class-based context manager
class Timer:
    """Measures execution time of a code block."""

    def __init__(self, label: str) -> None:
        self.label = label
        self.elapsed: float = 0.0

    def __enter__(self) -> "Timer":
        self._start = time.perf_counter()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb) -> bool:
        self.elapsed = time.perf_counter() - self._start
        print(f"  [{self.label}] Elapsed: {self.elapsed:.4f}s")
        return False  # Don't suppress exceptions


print("Class-based context manager:")
with Timer("sum computation") as t:
    total = sum(range(1_000_000))
    print(f"  Sum = {total}")


# Function-based context manager using @contextmanager
@contextmanager
def temporary_state(obj: dict, **overrides):
    """Temporarily override dict values, then restore them."""
    original = {k: obj.get(k) for k in overrides}
    obj.update(overrides)
    try:
        yield obj
    finally:
        # Restore original values
        for k, v in original.items():
            if v is None:
                obj.pop(k, None)
            else:
                obj[k] = v


config = {"debug": False, "log_level": "INFO"}
print(f"\nBefore: {config}")

with temporary_state(config, debug=True, log_level="DEBUG") as cfg:
    print(f"Inside: {cfg}")

print(f"After:  {config}")


# Context manager for transaction-like behavior
@contextmanager
def transaction(data: list):
    """Rolls back list changes if an exception occurs."""
    snapshot = data.copy()
    try:
        yield data
    except Exception:
        data.clear()
        data.extend(snapshot)
        print("  Transaction rolled back!")
        raise


items = [1, 2, 3]
print(f"\nBefore transaction: {items}")

try:
    with transaction(items) as data:
        data.append(4)
        data.append(5)
        print(f"During transaction: {data}")
        raise ValueError("Something went wrong!")
except ValueError:
    pass

print(f"After failed transaction: {items}")

## When Patterns Help vs When They Over-Engineer

Design patterns are tools, not goals. Use them when they solve a real problem. Overusing
patterns leads to unnecessary complexity. Here are guidelines for when patterns are
appropriate versus when simpler Python idioms suffice.

In [ ]:
# OVER-ENGINEERED: Strategy pattern with full class hierarchy
from abc import ABC, abstractmethod


class GreetingStrategy(ABC):
    @abstractmethod
    def greet(self, name: str) -> str: ...


class FormalGreeting(GreetingStrategy):
    def greet(self, name: str) -> str:
        return f"Good day, {name}."


class CasualGreeting(GreetingStrategy):
    def greet(self, name: str) -> str:
        return f"Hey {name}!"


class Greeter:
    def __init__(self, strategy: GreetingStrategy) -> None:
        self._strategy = strategy

    def greet(self, name: str) -> str:
        return self._strategy.greet(name)


# That's 4 classes for something trivial!
print("Over-engineered:")
print(f"  {Greeter(FormalGreeting()).greet('Alice')}")


# JUST RIGHT: Use a simple function
def greet(name: str, style: str = "casual") -> str:
    styles = {
        "formal": f"Good day, {name}.",
        "casual": f"Hey {name}!",
    }
    return styles.get(style, styles["casual"])


print("\nJust right:")
print(f"  {greet('Alice', 'formal')}")
print(f"  {greet('Bob')}")


# WHEN PATTERNS DO HELP:
# - Multiple implementations that change at runtime (Strategy)
# - Decoupling event producers from consumers (Observer)
# - Complex object construction with validation (Builder)
# - Undo/redo functionality (Command)
# - Integrating third-party code with different APIs (Adapter)

print("\nWhen to use patterns:")
print("  + When you have 3+ interchangeable implementations")
print("  + When components need loose coupling")
print("  + When you need undo/redo or action queuing")
print("  + When wrapping incompatible third-party APIs")
print("\nWhen to skip patterns:")
print("  - When a simple function or dict does the job")
print("  - When you only have one implementation")
print("  - When adding the pattern is more code than the problem")
print("  - When YAGNI (You Aren't Gonna Need It) applies")

## Summary

### Behavioral Patterns in Python
- **Strategy**: Use first-class functions instead of class hierarchies; pass `Callable` parameters
- **Observer**: Implement with an `EventBus` using `subscribe()` and `emit()` with handler dicts
- **Iterator**: Implement `__iter__` and `__next__`; or use generators with `yield` for simplicity
- **Command**: Encapsulate actions as objects with `execute()` and `undo()` methods
- **Template Method**: Define the algorithm skeleton in an ABC; subclasses override specific steps

### Python-Specific Tools
- **Protocol**: Structural typing that formalizes duck typing with type checker support
- **Context managers**: `with` statement provides built-in resource management (RAII pattern)
- **`@contextmanager`**: Create context managers from generator functions

### Guidelines
- Start with the simplest solution (function, dict, module)
- Add patterns when complexity demands it (3+ implementations, decoupling, undo/redo)
- Prefer Python idioms: first-class functions, generators, context managers, protocols
- Avoid premature abstraction - let patterns emerge from real requirements